In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# **Training Face Mask Detection Model**

In [2]:
!pip install -q keras

In [3]:
import keras

Using TensorFlow backend.


In [4]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import argparse
import os
from imutils import paths


In [5]:
import tensorflow

In [6]:
trainImages = list(paths.list_images('/content/drive/My Drive/Colab Notebooks/cell/Train'))

In [7]:
testImages = list(paths.list_images('/content/drive/My Drive/Colab Notebooks/cell/Test'))

In [8]:
#importing images into file
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
train = []
labels = []
for i in trainImages: 
  label = i.split(os.path.sep)[-2]
  img = load_img(i, target_size = (224, 224), interpolation = "nearest", color_mode = "rgb")
  img = img_to_array(img)
  img = preprocess_input(img)
  train.append(img)
  labels.append(label)


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [9]:
#Loading the images into numpy array
train = np.array(train)
labels = np.array(labels)

In [10]:
#Converting labels numpy array as a categorical data
from sklearn.preprocessing import LabelBinarizer
from keras.utils.np_utils import to_categorical   
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [11]:
#Spliting the data into train and test datasets
from sklearn.model_selection import train_test_split
X_Train, X_Test, y_train, y_test = train_test_split(train, labels, train_size = 0.75, random_state = 40, stratify = labels)

In [12]:
print(X_Train.shape, X_Test.shape, y_train.shape, y_test.shape)

(3047, 224, 224, 3) (1016, 224, 224, 3) (3047, 2) (1016, 2)


In [13]:
#Image Data Generator is used to generate different images from the train data while passing multiple times to model
IDG = ImageDataGenerator(rotation_range = 50, width_shift_range = 0.5, height_shift_range = 0.5, horizontal_flip = True, vertical_flip = True, fill_mode = 'nearest')

In [14]:
#Input is passed to Mobilevnet2 algorithm to train the model
from tensorflow.keras.applications import MobileNetV2
base = MobileNetV2(alpha = 1.0, include_top = False, weights = 'imagenet',input_tensor = Input(shape = (224, 224, 3)), classes = 2)

9412608/9406464 [==============================] - 0s 0us/step


In [15]:
#Second layer of the model is being given layers to train
finalModel = base.output
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense
finalModel = AveragePooling2D(pool_size=(7, 7))(finalModel)
finalModel = Flatten(name = "flatten")(finalModel)
finalModel = Dense(128, activation = "relu")(finalModel)
finalModel = Dropout(0.5)(finalModel)
finalModel = Dense(2, activation = "softmax")(finalModel)

In [16]:
#Adding first layer and second layer of the models
model = Model(inputs = base.input, outputs = finalModel)

In [17]:
#Making the base layers as non trainable
for layers in base.layers:
  layers.trainable = False

In [18]:
#Setting learning rate, epochs (the number of times the data is passed to model), and batch size
INIT_LR = 1e-4
EPOCHS = 40
BS = 32
#Adamax is a optimizer which is used to optimize the model
from tensorflow.keras.optimizers import Adamax
model.compile(loss = "categorical_crossentropy", optimizer = Adamax(lr = INIT_LR, decay = INIT_LR // EPOCHS), metrics = ["accuracy"])


In [19]:
#Fitting the main model on the train data
main = model.fit(IDG.flow(X_Train, y_train, batch_size = BS), validation_data = (X_Test, y_test), validation_steps = len(X_Test) // BS, epochs = EPOCHS, steps_per_epoch = len(X_Train) // BS)

Epoch 1/40
95/95 [==============================] - 139s 1s/step - loss: 0.6835 - accuracy: 0.6441 - val_loss: 0.4032 - val_accuracy: 0.8622
Epoch 2/40
95/95 [==============================] - 138s 1s/step - loss: 0.5063 - accuracy: 0.7549 - val_loss: 0.2972 - val_accuracy: 0.9272
Epoch 3/40
95/95 [==============================] - 137s 1s/step - loss: 0.4217 - accuracy: 0.8109 - val_loss: 0.2255 - val_accuracy: 0.9587
Epoch 4/40
95/95 [==============================] - 138s 1s/step - loss: 0.3703 - accuracy: 0.8368 - val_loss: 0.1906 - val_accuracy: 0.9656
Epoch 5/40
95/95 [==============================] - 138s 1s/step - loss: 0.3435 - accuracy: 0.8491 - val_loss: 0.1680 - val_accuracy: 0.9705
Epoch 6/40
95/95 [==============================] - 137s 1s/step - loss: 0.3303 - accuracy: 0.8624 - val_loss: 0.1479 - val_accuracy: 0.9705
Epoch 7/40
95/95 [==============================] - 138s 1s/step - loss: 0.3025 - accuracy: 0.8760 - val_loss: 0.1332 - val_accuracy: 0.9724
Epoch 8/40
95

In [20]:
#Predicting the output on test data from the trained model
y_pred = model.predict(x = X_Test, batch_size = BS)

In [21]:
#Setting the value as maximum that is depending on the percentage of mask present predicting it as mask present or not present
y_pred = np.argmax(y_pred, axis=1)

In [22]:
#Setting the value as maximum that is depending on the percentage of mask present predicting it as mask present or not present
y_test = y_test.argmax(axis = 1)

In [23]:
#Predicting accuracy
from sklearn.metrics import r2_score
r_squared = r2_score(y_test, y_pred)

In [24]:
print(r_squared)

0.9251782945736434


In [25]:
#Saving the model
model.save('finalver.h5')